Check Asset Compliance: SINCE policy
================================


In [ ]:
# Define a compliance policy that alerts when an asset has expired.

# Main function parses in a url to the Archivist and client credentials , which is
# a user authorization. The main function would initialize an archivist connection
# using the url and the credentials, called "arch", then call arch.access_policies.list()
# with suitable properties and attributes.

In [ ]:
from json import dumps as json_dumps
from os import getenv
from time import sleep
from uuid import uuid4
from warnings import filterwarnings

from archivist.archivist import Archivist
from archivist.compliance_policy_requests import (
    CompliancePolicySince,
)
from archivist.logger import set_logger

filterwarnings("ignore", message="Unverified HTTPS request")

In [ ]:
URL = "https://app.rkvst.io"
URL = "https://app.dev-paul-0.wild.jitsuin.io"

In [ ]:
def create_compliance_policy(arch, tag):
    """Compliance policy which expires 10 seconds after a
    Maintenance Performed event on a 'Traffic Light' has occurred.

    Usually the expiry time is on the order of days or weeks..

    Additionally the use of tag is simply to make this example
    repeatable.
    """
    compliance_policy = arch.compliance_policies.create(
        CompliancePolicySince(
            description="Maintenance should be performed every 10 seconds",
            display_name="Regular Maintenance of Traffic light",
            asset_filter=[
                ["attributes.arc_display_type=Traffic Light"],
            ],
            event_display_type=f"Maintenance Performed {tag}",
            time_period_seconds=10,  # very short so we can test
        )
    )
    print("SINCE_POLICY:", json_dumps(compliance_policy, indent=4))
    return compliance_policy

In [ ]:
def create_traffic_light(arch):
    """
    Creates a traffic light.

    Note that arc_display_type siginfies a Traffic Light
    """

    traffic_light = arch.assets.create(
        attrs={
            "arc_display_name": "Traffic light model 54",
            "arc_description": "Traffic flow control light at A603 North East",
            "arc_display_type": "Traffic Light",
        },
        confirm=True,
    )
    print("TRAFFIC_LIGHT:", json_dumps(traffic_light, indent=4))
    return traffic_light

In [ ]:
def perform_maintenance(arch, traffic_light, tag):
    """
    Perform maintenance on traffic light
    """
    maintenance_performed = arch.events.create(
        traffic_light["identity"],
        {
            "operation": "Record",
            "behaviour": "RecordEvidence",
        },
        {
            "arc_description": "Maintenance performed on traffic light",
            "arc_display_type": f"Maintenance Performed {tag}",
        },
        confirm=True,
    )
    print("MAINTENANCE_PERFORMED:", json_dumps(maintenance_performed, indent=4))

In [ ]:
"""Main function of create event.

Parse in user input of url and auth token and use them to
create an example archivist connection and create an asset.

"""
# optional call to set the logger level for all subsystems. The argument can
# be either "INFO" or "DEBUG". For more sophisticated logging control see the
# documentation.
set_logger("INFO")

# Get authorization token. The token grants certain rights and access permissions.
# The token can represent the root principal or user in an organization. Different tokens
# could indicate different users in the same organization or membership of different
# organisations.

with open("/home/builder/credentials/token", mode="r", encoding="utf-8") as tokenfile:
    auth = tokenfile.read().strip()

# Initialize connection to Archivist
arch = Archivist(
    URL,
    auth,
    max_time=300,
    verify=False,
)

In [ ]:
tag = uuid4()  # make this example repeatable

In [ ]:
# make a SINCE compliance policy that alerts when the
# maintenance performed event has expired.
compliance_policy = create_compliance_policy(arch, tag)

In [ ]:
# create an asset that matches the assets_filter field in the
# compliance policy.
traffic_light = create_traffic_light(arch)

In [ ]:
# perform maintenance on the asset which is valid for 10 seconds.
perform_maintenance(arch, traffic_light, tag)

# and check compliance - should be OK.
print("Sleep 1 second...")
sleep(1)
compliance = arch.compliance.compliant_at(
    traffic_light["identity"],
)
print("COMPLIANCE (true):", json_dumps(compliance, indent=4))

# however waiting long enough (> 10s) will cause the asset to
# become non-compliant...
print("Sleep 15 seconds...")
sleep(15)
compliance = arch.compliance.compliant_at(
    traffic_light["identity"],
)
print("COMPLIANCE (false):", json_dumps(compliance, indent=4))

In [ ]:
# finally delete the compliance_policy
_ = arch.compliance_policies.delete(
    compliance_policy["identity"],
)